### Install Packages

In [0]:
%pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.0.344 llama-index==0.9.3 databricks-vectorsearch==0.22 pydantic==1.10.9 databricks-sdk==0.12.0 mlflow[databricks] protobuf==3.19.0 
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached unstructured-0.10.30-py3-none-any.whl (1.7 MB)
  Using cached langchain-0.0.344-py3-none-any.whl (1.9 MB)
  Using cached llama_index-0.9.3-py3-none-any.whl (886 kB)
  Using cached databricks_vectorsearch-0.22-py3-none-any.whl (8.5 kB)
  Using cached pydantic-1.10.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached databricks_sdk-0.12.0-py3-none-any.whl (301 kB)
  Using cached mlflow-2.15.1-py3-none-any.whl (26.3 MB)
  Using cached protobuf-3.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached lxml-5.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
  Using cached python_iso639-2024.4.27-py3-none-any.whl (274 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached

### Enter Credentials

In [0]:
# dbutils.widgets.text("azure_search_endpoint","")
# dbutils.widgets.text("primary_admin_key","")
# dbutils.widgets.text("azure_openai_endpoint","")
# dbutils.widgets.text("azure_openai_key","")

### Load Sample Data 

In [0]:
import json

file_path = '/dbfs/FileStore/tables/text_sample.json'
with open(file_path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

In [0]:
from mlflow.deployments import get_deploy_client

deploy_client = get_deploy_client("databricks")

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]

In [0]:
title_response = deploy_client.predict(endpoint="textEmbeddingOpenAImodel", inputs={"input": titles})
print(title_response.data)

[{'object': 'embedding', 'index': 0, 'embedding': [-0.03197575, -0.0031608075, 0.028109245, 0.0055477214, 0.016841825, -0.050240826, -0.00048998446, 0.019818796, 0.0028272325, -0.0035818531, 0.0064461497, -0.049956176, 0.039352942, -0.018502288, -0.044808745, 0.016877407, -0.03503574, 0.041392937, -0.018976705, -0.016580896, 0.004109643, 0.00045254995, -0.027350176, 0.031192958, -0.06034592, 0.018775078, -0.020933678, -0.029366452, -0.055364538, -0.070261255, 0.02395809, -0.035747368, 0.027397618, 0.016367408, 0.047536645, 0.023566695, 0.017707638, -0.005965802, 0.0073831244, 0.0166402, -0.0071103345, 0.015442294, -0.009114749, 0.062291034, 0.023803905, 0.065232426, -0.029010639, -0.032141794, 0.0029399067, 0.022748325, -0.033565048, -0.015157644, -0.0033446443, -0.0060666157, -0.016521594, 0.018715775, -0.004204526, 0.031667378, 0.015904851, 0.018466705, -0.011789278, -0.04148782, -0.022024838, 0.070830554, -0.013117648, 0.003002174, 0.0056277793, 0.0093341665, -0.012951601, -0.017980

In [0]:
title_embeddings = [item['embedding'] for item in title_response.data]
print(title_embeddings)

[[-0.03197575, -0.0031608075, 0.028109245, 0.0055477214, 0.016841825, -0.050240826, -0.00048998446, 0.019818796, 0.0028272325, -0.0035818531, 0.0064461497, -0.049956176, 0.039352942, -0.018502288, -0.044808745, 0.016877407, -0.03503574, 0.041392937, -0.018976705, -0.016580896, 0.004109643, 0.00045254995, -0.027350176, 0.031192958, -0.06034592, 0.018775078, -0.020933678, -0.029366452, -0.055364538, -0.070261255, 0.02395809, -0.035747368, 0.027397618, 0.016367408, 0.047536645, 0.023566695, 0.017707638, -0.005965802, 0.0073831244, 0.0166402, -0.0071103345, 0.015442294, -0.009114749, 0.062291034, 0.023803905, 0.065232426, -0.029010639, -0.032141794, 0.0029399067, 0.022748325, -0.033565048, -0.015157644, -0.0033446443, -0.0060666157, -0.016521594, 0.018715775, -0.004204526, 0.031667378, 0.015904851, 0.018466705, -0.011789278, -0.04148782, -0.022024838, 0.070830554, -0.013117648, 0.003002174, 0.0056277793, 0.0093341665, -0.012951601, -0.017980428, -0.03014924, 0.0021422922, 0.00221642, -0.02

In [0]:
embedding_dimensions = len(title_embeddings[0])
print(embedding_dimensions)

1536


In [0]:
content_response = deploy_client.predict(endpoint="textEmbeddingOpenAImodel", inputs={"input": content[0]})
content_response_list = content_response.data[0]['embedding']
print(content_response_list)

[-0.018428253, -0.013230006, 0.04283605, 0.0075056846, 0.041731816, -0.047544684, -0.02810595, -0.00378149, 0.013209172, 0.027668422, 0.02331398, -0.048753094, 0.0130529115, -0.021136759, -0.032856252, 0.02756425, -0.027814265, 0.035856444, -0.0099798, -0.045002855, 0.011979927, -0.01964708, -0.028376801, 0.040606745, -0.05500349, -0.01025065, -0.025293272, -0.0067191767, -0.059462108, -0.07825497, -0.0026954836, -0.017907387, -0.0031590548, 0.04537788, 0.05267001, -0.005765991, 0.023230642, -0.02989773, 0.06321234, 0.010969446, -0.03489805, 0.015990598, -0.00978708, 0.041231785, -0.010771517, 0.058003683, -0.00990167, -0.04666963, 0.0065004127, 0.021647207, -0.045169532, -0.014292574, 0.017407356, -0.0052972115, 0.0022488406, 0.01894912, 0.0048128054, 0.03902331, -0.015323889, 0.03789824, 0.003922124, 0.00018067553, -0.028272629, 0.08096347, -0.047002982, 0.009453725, -0.009068284, -0.0017201613, -0.030793622, -0.004401321, -0.017407356, -0.02187639, 0.007906752, -0.007458807, 0.00510

In [0]:
import time

content_embeddings = []

for i in range(len(content)):
    content_response = deploy_client.predict(endpoint="textEmbeddingOpenAImodel", inputs={"input": content[i]})
    content_response_list = content_response.data[0]['embedding']
    content_embeddings.append(content_response_list)
    print(len(content_response_list))
    print(len(content_embeddings))
    print('-------')
    time.sleep(1)

print(content_embeddings)

1536
1
-------
1536
2
-------
1536
3
-------
1536
4
-------
1536
5
-------
1536
6
-------
1536
7
-------
1536
8
-------
1536
9
-------
1536
10
-------
1536
11
-------
1536
12
-------
1536
13
-------
1536
14
-------
1536
15
-------
1536
16
-------
1536
17
-------
1536
18
-------
1536
19
-------
1536
20
-------
1536
21
-------
1536
22
-------
1536
23
-------
1536
24
-------
1536
25
-------
1536
26
-------
1536
27
-------
1536
28
-------
1536
29
-------
1536
30
-------
1536
31
-------
1536
32
-------
1536
33
-------
1536
34
-------
1536
35
-------
1536
36
-------
1536
37
-------
1536
38
-------
1536
39
-------
1536
40
-------
1536
41
-------
1536
42
-------
1536
43
-------
1536
44
-------
1536
45
-------
1536
46
-------
1536
47
-------
1536
48
-------
1536
49
-------
1536
50
-------
1536
51
-------
1536
52
-------
1536
53
-------
1536
54
-------
1536
55
-------
1536
56
-------
1536
57
-------
1536
58
-------
1536
59
-------
1536
60
-------
1536
61
-------
1536
62
-------
1536
63
-------
1

In [0]:
# Generate embeddings for title and content fields
for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

print(input_data)

[{'id': '1', 'title': 'Azure App Service', 'content': 'Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.', 'category': 'Web', 'titleVector': [-0.03197575, -0.0031608075, 0.028109245, 0.0055477214, 0.016841825, -0.050240826, -0.00048998446, 0.019818796, 0.0028272325, -0.0035818531, 0.0064461497, -0.049956176, 0.039352942, -0.018502288, -0.044808745, 0.016877407, -0.03503574, 0.041392937, -0.018976705, -0.016580896, 0.004109643, 0.00045254995, -0.027350176, 0.031192958, -0.06034592, 0.018775078, -0.020933678, -0.029366452, -0.055364538, -0.070261255, 0.02395809, -0.035747368, 0.027397618, 0.016367408, 0.04

In [0]:
# Output embeddings to docVectors.json file
output_path = '/dbfs/FileStore/tables/docVectors.json'
with open(output_path, 'w') as f:
    json.dump(input_data, f)

In [0]:
import json

output_path = '/dbfs/FileStore/tables/docVectors.json'
with open(output_path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

print(input_data)

[{'id': '1', 'title': 'Azure App Service', 'content': 'Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.', 'category': 'Web', 'titleVector': [-0.03197575, -0.0031608075, 0.028109245, 0.0055477214, 0.016841825, -0.050240826, -0.00048998446, 0.019818796, 0.0028272325, -0.0035818531, 0.0064461497, -0.049956176, 0.039352942, -0.018502288, -0.044808745, 0.016877407, -0.03503574, 0.041392937, -0.018976705, -0.016580896, 0.004109643, 0.00045254995, -0.027350176, 0.031192958, -0.06034592, 0.018775078, -0.020933678, -0.029366452, -0.055364538, -0.070261255, 0.02395809, -0.035747368, 0.027397618, 0.016367408, 0.04

In [0]:
azure_search_endpoint = dbutils.widgets.get("azure_search_endpoint")
primary_admin_key = dbutils.widgets.get("primary_admin_key")
azure_openai_endpoint = dbutils.widgets.get("azure_openai_endpoint")
azure_openai_key = dbutils.widgets.get("azure_openai_key")

In [0]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient

from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)

# Create a search index
azure_search_endpoint = azure_search_endpoint
primary_admin_key = primary_admin_key
azure_openai_endpoint = azure_openai_endpoint
azure_openai_embedding_deployment = "textEmbeddingOpenAImodel"
embedding_model_name = "text-embedding-3-small"
azure_openai_key = azure_openai_key
credential = AzureKeyCredential(primary_admin_key)

index_name = "vector-search-index-01"

index_client = SearchIndexClient(endpoint=azure_search_endpoint, credential=credential)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=embedding_dimensions, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=embedding_dimensions, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="myHnsw")
    ],
    profiles=[
        VectorSearchProfile(name="myHnswProfile", algorithm_configuration_name="myHnsw", vectorizer="myVectorizer")
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myVectorizer",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=azure_openai_endpoint,
                deployment_id=azure_openai_embedding_deployment,
                model_name=embedding_model_name,
                api_key=azure_openai_key
            )
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 vector-search-index-01 created


In [0]:
from azure.search.documents import SearchClient

search_client = SearchClient(endpoint=azure_search_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(input_data)
print(f"Uploaded {len(input_data)} documents") 

Uploaded 108 documents


In [0]:
from azure.search.documents.models import VectorizedQuery
from mlflow.deployments import get_deploy_client

deploy_client = get_deploy_client("databricks")

query = "Kubernetes"  

response = deploy_client.predict(endpoint="textEmbeddingOpenAImodel", inputs={"input": query}).data[0]
embedding = response['embedding']

vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure Kubernetes Service (AKS)
Score: 0.68138754
Content: Azure Kubernetes Service (AKS) is a managed container orchestration service based on the popular open-source Kubernetes system. It simplifies Kubernetes deployment and management, making it easy for developers and administrators to deploy, scale, and manage containerized applications. AKS offers automatic upgrades, scaling, and self-healing capabilities, reducing the operational overhead of managing Kubernetes clusters. It also integrates with Azure services like Azure Active Directory, Azure Monitor, and Azure Policy, providing a seamless experience for managing your applications and infrastructure.
Category: Containers

Title: Azure Kubernetes Service (AKS)
Score: 0.6744978
Content: Azure Kubernetes Service (AKS) is a fully managed, container orchestration platform that enables you to deploy, scale, and manage containerized applications using Kubernetes. It provides features like automatic scaling, rolling updates, and 